In [1]:
import pandas as pd
import numpy as np

# Human Resource / People analytics data.
Project for data cleaning and data visualisation.

# Data cleaning

## Extraction Phase

There is a total of 5 data in which one of them is about survey. The remaining 4 data is considered to essential for the overall analysis while the survey data will be separately used for engagement analysis. Both combined will create People Analytics.

I identified the 4 data as listed:
- company data → `2021.06_COL_2021.txt`
- job_detail data → `2021.06_job_profile_mapping.txt`
- fact data → `CompanyData.txt`
- demographic data → `Diversity.txt`

In addition with survey data → `EngagementSurvey.txt`

### Importing first 4 data

In [3]:
company_data = pd.read_csv('data/2021.06_COL_2021.txt', sep='\t')
company_data.head()

,Office,COL Amount,Currency
0,NYC,100,USD
1,Boulder,70,USD
2,Oslo,70,NOK
3,SanJose,90,USD
4,London,90,GBP


In [5]:
job_data = pd.read_csv('data/2021.06_job_profile_mapping.txt', sep='\t')
job_data.head()

,Department,Job_title,Job_Profile,Compensation,Level,Bonus %
0,Corporate,CEO,JP_1000,"500,000.00",CSuite,1.00
1,Corporate,HR Manager,JP_1001,"100,000.00",Manager,0.20
2,Corporate,AR Specialist,JP_1002,"65,000.00",Individual Contributor,0.15
3,Corporate,AP Specialist,JP_1003,"65,000.00",Individual Contributor,0.15
4,Corporate,FP&A Analyst,JP_1004,"70,000.00",Individual Contributor,0.15


In [7]:
full_data = pd.read_csv('data/CompanyData.txt', sep='\t', encoding='utf_16_le')
full_data.head()

,EmployeeID,First_Name,Surname,StreetAddress,City,State,StateFull,ZipCode,Country,CountryFull,...,Department,Currency,Bonus_pct,Job_title,DOB,level,Salary,Active Status,Job_Profile,Notes
0,100001,Patrice,Moore,1427 Buckhannan Avenue,North Syracuse,NY,New York,13212,US,United States,...,Corporate,USD,1.00,CEO,1/5/1986,CSuite,500000,1,JP_1000,Changes for 2021.06:
1,100002,David,Rickards,4265 Graystone Lakes,Macon,GA,Georgia,31206,US,United States,...,Corporate,USD,0.20,HR Manager,7/13/1971,Manager,70000,1,JP_1001,Changes for 2021.06:
2,100003,Grace,Maldonado,1680 Hudson Street,Weehawken,NJ,New Jersey,07087,US,United States,...,Marketing,USD,0.15,Graphic Designer,1/25/1989,Individual Contributor,77000,0,JP_1022,Changes for 2021.06:
3,100004,Justin,Edgin,1262 Limer Street,Rome,GA,Georgia,30165,US,United States,...,Technology,USD,0.50,CTO,5/1/1996,CSuite,400000,0,JP_1036,Changes for 2021.06:
4,100005,Benjamin,Vargas,2431 Rainbow Road,Santa Ana,CA,California,92705,US,United States,...,Customer Service,USD,0.15,Associate Account Manager,5/5/1972,Manager,51000,0,JP_1015,Changes for 2021.06:


In [8]:
demographic_data = pd.read_csv('data/Diversity.txt', sep='\t')
demographic_data.head()

,EmployeeID,Gender,Gender Identity,Race/Ethnicity,Veteran,Disability,Education,Sexual Orientation
0,100001,female,female,White,0,0,Undergraduate,Heterosexual
1,100002,male,male,White,0,1,Undergraduate,Heterosexual
2,100003,female,female,Asian,0,0,Undergraduate,Heterosexual
3,100004,male,male,White,0,0,Undergraduate,Heterosexual
4,100005,male,male,Hispanic or Latino,0,0,Undergraduate,Missing
